In [ ]:
!pip install razdel

In [ ]:
import json

import pandas as pd
from transformers import BertModel, BertTokenizerFast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/diploma/data

In [ ]:
from utils import clean_text, is_text_valid

In [ ]:
from align_sentences import align_sentences

In [ ]:
DATA_PATH_PREFIX = 'drive/MyDrive/diploma/data/'

In [ ]:
MODEL_PATH = 'drive/MyDrive/diploma/labse_moksha_40k+5k'

# Load df with parallel texts

In [ ]:
with open(DATA_PATH_PREFIX + 'e-mordovia/aligned_news_texts_09_02.json', 'r') as f:
    e_mordovia_texts_df = pd.DataFrame(json.load(f))

In [ ]:
e_mordovia_texts_df.shape

(4560, 2)

In [ ]:
e_mordovia_texts_df.head()

mdf  \
0  Кельгома выпускникт!\nТячи тинь эряфсонтт инь ...   
1  «Дорожная сеть» региональнай проектть рамкаса ...   
2  Мордовиянь Оцюнясь Владимир Волков 22 январьст...   
3  Мордовиянь Оцюнясь Владимир Волков етафтсь раб...   
4  Еткшиста, 8 апрельста, Мордовиянь Оцюнясь Влад...   

                                                  ru  
0  Дорогие выпускники!\nСегодня один из самых зна...  
1  В рамках регионального проекта «Дорожная сеть»...  
2  Глава Мордовии Владимир Волков 22 января прове...  
3  Глава Мордовии Владимир Волков провел рабочую ...  
4  В субботу, 8 апреля, Глава Мордовии Владимир В...

# Align each document

In [ ]:
model = BertModel.from_pretrained(MODEL_PATH)
tokenizer = BertTokenizerFast.from_pretrained(MODEL_PATH)

In [ ]:
model.cuda();

In [ ]:
all_aligned_pairs = []

for idx, (mdf_text, ru_text) in enumerate(zip(
    e_mordovia_texts_df['mdf'],
    e_mordovia_texts_df['ru']
)):
    if not ru_text or not mdf_text:
        print(f"empty pair: ({idx}), {mdf_text}, {ru_text}")
        continue

    aligned_pairs = align_sentences(mdf_text, ru_text, model, tokenizer)
    all_aligned_pairs += aligned_pairs

    if not aligned_pairs:
        print(f"0 aligned pairs: {idx}")

0 aligned pairs: 2841
0 aligned pairs: 2843
0 aligned pairs: 3511
0 aligned pairs: 3898
0 aligned pairs: 3928
0 aligned pairs: 4132
0 aligned pairs: 4199
0 aligned pairs: 4248
0 aligned pairs: 4264
0 aligned pairs: 4271
0 aligned pairs: 4283
0 aligned pairs: 4294
0 aligned pairs: 4319
0 aligned pairs: 4334
0 aligned pairs: 4345
0 aligned pairs: 4350
0 aligned pairs: 4351
0 aligned pairs: 4360
0 aligned pairs: 4391
0 aligned pairs: 4396
0 aligned pairs: 4417
0 aligned pairs: 4421
0 aligned pairs: 4422
0 aligned pairs: 4446
0 aligned pairs: 4455
0 aligned pairs: 4475
0 aligned pairs: 4484
0 aligned pairs: 4491
0 aligned pairs: 4494
0 aligned pairs: 4498
0 aligned pairs: 4503
0 aligned pairs: 4504
0 aligned pairs: 4510
0 aligned pairs: 4511
0 aligned pairs: 4528
0 aligned pairs: 4532


# Save all parallel pairs

In [ ]:
data = []
for mdf, ru in all_aligned_pairs:
    cleaned_mdf = clean_text(mdf)
    cleaned_ru = clean_text(ru)

    if not is_text_valid(cleaned_mdf) or not is_text_valid(cleaned_ru):
        continue

    data.append({'mdf': cleaned_mdf, 'ru': cleaned_ru})

In [ ]:
len(data)

65970

In [ ]:
with open(DATA_PATH_PREFIX + 'aligned_news_sents_09_02.json', "w") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)